In [ ]:
import numpy as np
import datetime
import xarray as xr

from tools.utils import create_polydict_ptsarr, xy2ll, ll2xy
from tools.itslive_utils import compute_stress_strain


import matplotlib.pyplot as plt
import cmcrameri as cmc
from tools.plot_tools import plot_latlon_lines, shifted_colormap
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [ ]:
import plotSettings
plotSettings.paper()
COLS = plotSettings.COLS

In [ ]:
%matplotlib tk

In [ ]:
# general data directory
data_dir = '/'

In [ ]:
center_lon, center_lat = 69.5881, -72.0370
center_x, center_y = ll2xy(center_lon, center_lat)

# bounding boxes for masking images 
bounding_box_file = 'GIS/BoundingBoxes2.shp'
bbs_dict, _ = create_polydict_ptsarr(os.path.join(data_dir, bounding_box_file))
bb = bbs_dict[0]

# doline outlines
doline_outline_file = 'GIS/DolineOutlines.shp'
do_dict, atr = create_polydict_ptsarr(os.path.join(data_dir, doline_outline_file))
doline = do_dict[0]

vel_pair_dir = os.path.join(data_dir, 'data/MEaSUREs_ITS_LIVE/velocity_pairs/2015')
vel_pair_files = ['LO08_L1GT_128111_20150302_20200924_02_T2_X_LC08_L1GT_128111_20150318_20200909_02_T2_G0120V02_P053.nc',
                  'LC08_L1GT_127111_20150223_20201016_02_T2_X_LC08_L1GT_127111_20150327_20200909_02_T2_G0120V02_P039.nc',
                  'LO08_L1GT_128111_20150302_20200924_02_T2_X_LC08_L1GT_128111_20150403_20200909_02_T2_G0120V02_P064.nc',
                  'LC08_L1GT_128111_20150318_20200909_02_T2_X_LC08_L1GT_128111_20150403_20200909_02_T2_G0120V02_P041.nc'
                 ]

In [ ]:
# convert EPSG3031 extent to lat lon extent
extent = bbs_dict[0].bounds
left, bottom, right, top = extent
ll = xy2ll(left, bottom), xy2ll(left, top), xy2ll(right, top), xy2ll(right, bottom)
ll = np.array(ll)
lon_range = [ll[:,0].min(), ll[:,0].max()]
lat_range = [ll[:,1].min(), ll[:,1].max()]

In [ ]:
SAVE = False
save_name = 'figure6'
save_dir = 'figs'
save_type = '.svg'

########################################################################################################
# some settings
########################################################################################################

NROWS = 2
NCOLS = 4

dt_fmt = '%Y%m%d'

########################################################################################################
# create figure
########################################################################################################
fig = plt.figure(num=1, clear=1, figsize=[15.88266667,  6.70933333])

PARENT = gridspec.GridSpec(NROWS, NCOLS, figure=fig,
                           hspace=0.03, wspace=0.001,
                           left=0.03, right=0.97,
                           bottom=0.03, top=0.97
                          )
gs0 = gridspec.GridSpecFromSubplotSpec(1, NCOLS, subplot_spec=PARENT[0,:])
strainmaps = [fig.add_subplot(gs0[i]) for i in range(NCOLS)]

gs1 = gridspec.GridSpecFromSubplotSpec(1, NCOLS, subplot_spec=PARENT[1,:])
stressmaps = [fig.add_subplot(gs1[i]) for i in range(NCOLS)]

########################################################################################################
# calculate stress and strain
########################################################################################################

for i, file in enumerate(vel_pair_files):
    file_name = file.split('/')[-1]
    first_date, second_date = file_name.split('_')[3], file_name.split('_')[11]
    first_date = datetime.datetime.strptime(first_date, dt_fmt)
    second_date = datetime.datetime.strptime(second_date, dt_fmt)
    title = '{0} $-$ {1}'.format(second_date.__str__()[:10], first_date.__str__()[:10])
    
    ds = xr.open_dataset(os.path.join(vel_pair_dir, file))
    
    strain_dict, stress_dict, D = compute_stress_strain(ds.vx.values, ds.vy.values,
                                                        dx=120, dy=-120,
                                                        grad_method='sgolay', window_size=31,# order=2,
                                                        inpaint=True, rotate=True)
    for k, v in strain_dict.items():
        ds[k] = (['y', 'x'], v)
    for k, v in stress_dict.items():
        ds[k] = (['y', 'x'], v)
    ds = ds.isel(x=((ds.x >= left) & (ds.x <= right)),
                 y=((ds.y >= bottom) & (ds.y <= top))
                 )
########################################################################################################
# plot strain
########################################################################################################
    strainmaps[i].set_title(title)
    
    vmin,vmax = np.nanmin(ds['e_xx']), np.nanmax(ds['e_xx'])
    scm = shifted_colormap(cmc.cm.broc, midpoint=1 - vmax/(vmax + abs(vmin)))
    p = strainmaps[i].contourf(ds.x, ds.y, ds['e_xx'],
                               cmap=scm,
                               rasterized=True)


    inset = strainmaps[i].inset_axes([0.1, 0.2, 0.4, 0.07])
    inset.xaxis.set_ticks_position('bottom')
    cb = fig.colorbar(p, cax=inset, orientation='horizontal', label=r'$\dot{{\epsilon}}_{{xx}}$ (1/yr)')

    ticks = [vmin, 0, vmax]
    cb.set_ticks(ticks)
    cb.set_ticklabels(['${0:0.2f}$'.format(v) for v in ticks])

    strainmaps[i].set(xticklabels=[], yticklabels=[],
                      xticks=[], yticks=[],
                      aspect='equal',
                      xlim=(left, right),
                      ylim=(bottom, top)
                     )
    
    strainmaps[i].plot(doline.exterior.xy[0], doline.exterior.xy[1], c='k', ls='--', lw=2)
    strainmaps[i].plot(bbs_dict[3].boundary.xy[0], bbs_dict[3].boundary.xy[1], c='k', ls='-', lw=1)
    
    scalebar = AnchoredSizeBar(strainmaps[i].transData,
                               1e3, '1 km', 'upper right', 
                               pad=1,
                               frameon=False,
                               size_vertical=100,
                              )
    
    strainmaps[i].add_artist(scalebar)
    
    strainmaps[i] = plot_latlon_lines(strainmaps[i],
                                      lat_range=lat_range, lon_range=lon_range,
                                      extents=(left, right, bottom, top),
                                      lon_text=True if i == 0 else False
                                     )

    [a.set_linewidth(1) for _,a in strainmaps[i].spines.items()]
    
########################################################################################################
# plot stress map
########################################################################################################
    vmin,vmax = np.nanmin(ds['t_xx']), np.nanmax(ds['t_xx'])
    scm = shifted_colormap(cmc.cm.broc, midpoint=1 - vmax/(vmax + abs(vmin)))
    p = stressmaps[i].contourf(ds.x, ds.y, ds['t_xx']/1e6,
                               cmap=scm, 
                               rasterized=True)


    inset = stressmaps[i].inset_axes([0.1, 0.2, 0.4, 0.07])
    inset.xaxis.set_ticks_position('bottom')
    cb = fig.colorbar(p, cax=inset, orientation='horizontal', label=r'$\sigma_{{xx}}$ (MPa)')

    ticks = [vmin / 1e6 , 0, vmax / 1e6]
    cb.set_ticks(ticks)
    cb.set_ticklabels(['${0:0.1f}$'.format(v) for v in ticks])

    stressmaps[i].set(xticklabels=[], yticklabels=[],
                      xticks=[], yticks=[],
                      aspect='equal',
                      xlim=(left, right),
                      ylim=(bottom, top)
                     )

    stressmaps[i].plot(doline.exterior.xy[0], doline.exterior.xy[1], c='k', ls='--', lw=2)
    stressmaps[i].plot(bbs_dict[3].boundary.xy[0], bbs_dict[3].boundary.xy[1], c='k', ls='-', lw=1)
    
    scalebar = AnchoredSizeBar(stressmaps[i].transData,
                               1e3, '1 km', 'upper right', 
                               pad=1,
                               frameon=False,
                               size_vertical=100,
                              )
    
    stressmaps[i].add_artist(scalebar)
    
    stressmaps[i] = plot_latlon_lines(stressmaps[i],
                                      lat_range=lat_range, lon_range=lon_range,
                                      extents=(left, right, bottom, top),
                                      lon_text=True if i == 0 else False,
                                      lat_text=False
                                     )

    [a.set_linewidth(1) for _,a in stressmaps[i].spines.items()]
    
    plt.pause(0.01)
    
if SAVE:
    plt.savefig(os.path.join(save_dir, save_name + fig_type), dpi=600.)